In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
vendor_name = 'SHAH KOTHARI BROS'
start_date_pn = '01-09-2024'
end_date_pn = '31-12-2024'

start_date_cn = '01-09-2024'
end_date_cn = '23-01-2025'

In [3]:
pn = pd.read_csv('pn_data.csv')
pn.head()

,vendor_id,vendor_name,pick_note_number,date,key,ws_code,batch_number,quantity,month
0,342,J.R.SHAH AND COMPANY,PR201008001,01-10-2024,342_19316_O309,19316,O309,1,Oct-2024
1,356,ATUL MEDICAL AGENCY,PR201008005,01-10-2024,356_4382_K2400513,4382,K2400513,1,Oct-2024
2,357,DHRUVI HEALTHCARE PVT LTD,PR201008007,01-10-2024,357_1091_BASM036,1091,BASM036,1,Oct-2024
3,358,B K DISTRIBUTORS,PR201007997,01-10-2024,358_23588_2C29L009,23588,2C29L009,85,Oct-2024
4,359,D K PHARMA,PR201008003,01-10-2024,359_5457_KGA2301A,5457,KGA2301A,1,Oct-2024


In [4]:
cn = pd.read_csv('cn_data.csv')
cn.head()

,vendor_cn_date,debit_note_number,key,billed_quantity,month,vendor_name
0,2024-09-01,DN201005468,388_10098_GEO37697,1,Sep-2024,PHARMA DISTRIBUTOR
1,2024-09-01,DN201005466,388_10779_LIL1L00,20,Sep-2024,PHARMA DISTRIBUTOR
2,2024-09-01,DN201005468,388_10779_LIL1L00,5,Sep-2024,PHARMA DISTRIBUTOR
3,2024-09-01,DN201005468,388_11141_230330040,1,Sep-2024,PHARMA DISTRIBUTOR
4,2024-09-01,DN201005468,388_11681_220910719,3,Sep-2024,PHARMA DISTRIBUTOR


In [5]:
pn['date'] = pd.to_datetime(pn['date'], format='%d-%m-%Y')
cn['vendor_cn_date'] = pd.to_datetime(cn['vendor_cn_date'])

In [6]:
# 3. Sort DataFrames
pn = pn.sort_values('date')
cn = cn.sort_values('vendor_cn_date')

### Version 1

In [1]:
import pandas as pd
from datetime import datetime
from collections import defaultdict

def process_mapping():
    # Load and prepare data
    pn = pd.read_csv('pn_data.csv')
    cn = pd.read_csv('cn_data.csv')
    
    pn['date'] = pd.to_datetime(pn['date'], format='%d-%m-%Y')
    cn['vendor_cn_date'] = pd.to_datetime(cn['vendor_cn_date'])
    
    pn = pn.sort_values('date')
    cn = cn.sort_values('vendor_cn_date')
    
    mapped_records = []
    cn_remaining = defaultdict(dict)  # Track remaining CN quantities by key and date
    
    # Process each key
    for key in pn['key'].unique():
        key_pns = pn[pn['key'] == key].copy()
        key_cns = cn[cn['key'] == key].copy()
        
        if key_cns.empty:
            continue
            
        # Initialize CN remainings for this key
        for _, cn_row in key_cns.iterrows():
            cn_remaining[key][cn_row['vendor_cn_date']] = cn_row['billed_quantity']
            
        pn_remaining = 0  # Track remaining PN quantity
        
        for _, pn_row in key_pns.iterrows():
            current_pn_qty = pn_row['quantity'] + pn_remaining
            
            # Find eligible CNs with remaining quantity
            eligible_cns = key_cns[
                (key_cns['vendor_cn_date'] >= pn_row['date']) & 
                (key_cns['vendor_cn_date'].map(lambda x: cn_remaining[key][x] > 0))
            ].copy()
            
            if eligible_cns.empty:
                pn_remaining = current_pn_qty
                continue
                
            nearest_cn = eligible_cns.iloc[0]
            cn_date = nearest_cn['vendor_cn_date']
            cn_qty_available = cn_remaining[key][cn_date]
            
            qty_to_map = min(current_pn_qty, cn_qty_available)
            
            mapped_records.append({
                'PN_Date': pn_row['date'],
                'CN_Date': cn_date,
                'Pick_Note': pn_row['pick_note_number'],
                'Credit_Note': nearest_cn['debit_note_number'],
                'Key': key,
                'Vendor': pn_row['vendor_name'],
                'PN_Qty': qty_to_map,
                'CN_Qty': qty_to_map,
                'PN_Remaining': current_pn_qty - qty_to_map,
                'CN_Remaining': cn_qty_available - qty_to_map,
                'Original_PN_Qty': pn_row['quantity'],
                'Original_CN_Qty': nearest_cn['billed_quantity']
            })
            
            # Update remaining quantities
            cn_remaining[key][cn_date] -= qty_to_map
            pn_remaining = current_pn_qty - qty_to_map
            
    result_df = pd.DataFrame(mapped_records)
    result_df['PN_Date'] = result_df['PN_Date'].dt.strftime('%Y-%m-%d')
    result_df['CN_Date'] = result_df['CN_Date'].dt.strftime('%Y-%m-%d')
    
    result_df.to_csv('p_c_r.csv', index=False)
    return result_df

if __name__ == "__main__":
    results = process_mapping()
    print("Mapping completed. Check p_c_r.csv")

Mapping completed. Check p_c_r.csv


### Version 2

In [1]:
import pandas as pd
from datetime import datetime
from collections import defaultdict

def process_mapping():
    # Load and prepare data
    pn = pd.read_csv('pn_data.csv')
    cn = pd.read_csv('cn_data.csv')
    
    
    pn['date'] = pd.to_datetime(pn['date'], format='%d-%m-%Y')
    cn['vendor_cn_date'] = pd.to_datetime(cn['vendor_cn_date'])
    
    pn = pn.sort_values('date')
    cn = cn.sort_values('vendor_cn_date')
    
    mapped_records = []
    used_cn_indices = defaultdict(set)
    
    # Process each key
    for key in pn['key'].unique():
        key_pns = pn[pn['key'] == key].copy()
        key_cns = cn[cn['key'] == key].copy()
        
        pn_remaining = 0
        
        # First pass: Map PNs with available CNs
        for idx, pn_row in key_pns.iterrows():
            current_pn_qty = pn_row['quantity'] + pn_remaining
            
            if not key_cns.empty:
                eligible_cns = key_cns[
                    (key_cns['vendor_cn_date'] >= pn_row['date']) & 
                    (~key_cns.index.isin(used_cn_indices[key]))
                ].copy()
                
                if not eligible_cns.empty:
                    nearest_cn = eligible_cns.iloc[0]
                    cn_qty_available = nearest_cn['billed_quantity']
                    
                    qty_to_map = min(current_pn_qty, cn_qty_available)
                    
                    mapped_records.append({
                        'PN_Date': pn_row['date'],
                        'CN_Date': nearest_cn['vendor_cn_date'],
                        'Pick_Note': pn_row['pick_note_number'],
                        'Credit_Note': nearest_cn['debit_note_number'],
                        'Key': key,
                        'Vendor': pn_row['vendor_name'],
                        'PN_Qty': qty_to_map,
                        'CN_Qty': qty_to_map,
                        'PN_Remaining': current_pn_qty - qty_to_map,
                        'CN_Remaining': cn_qty_available - qty_to_map,
                        'Original_PN_Qty': pn_row['quantity'],
                        'Original_CN_Qty': nearest_cn['billed_quantity']
                    })
                    
                    used_cn_indices[key].add(nearest_cn.name)
                    pn_remaining = current_pn_qty - qty_to_map
                    continue
            
            # Add unmapped PN record
            mapped_records.append({
                'PN_Date': pn_row['date'],
                'CN_Date': None,
                'Pick_Note': pn_row['pick_note_number'],
                'Credit_Note': None,
                'Key': key,
                'Vendor': pn_row['vendor_name'],
                'PN_Qty': current_pn_qty,
                'CN_Qty': 0,
                'PN_Remaining': current_pn_qty,
                'CN_Remaining': 0,
                'Original_PN_Qty': pn_row['quantity'],
                'Original_CN_Qty': 0
            })
        
        # Add unmapped CN records
        unmapped_cns = key_cns[~key_cns.index.isin(used_cn_indices[key])]
        for _, cn_row in unmapped_cns.iterrows():
            mapped_records.append({
                'PN_Date': None,
                'CN_Date': cn_row['vendor_cn_date'],
                'Pick_Note': None,
                'Credit_Note': cn_row['debit_note_number'],
                'Key': key,
                'Vendor': cn_row['vendor_name'],
                'PN_Qty': 0,
                'CN_Qty': cn_row['billed_quantity'],
                'PN_Remaining': 0,
                'CN_Remaining': cn_row['billed_quantity'],
                'Original_PN_Qty': 0,
                'Original_CN_Qty': cn_row['billed_quantity']
            })
    
    result_df = pd.DataFrame(mapped_records)
    
    # Format dates, handling None values
    result_df['PN_Date'] = result_df['PN_Date'].apply(
        lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else ''
    )
    result_df['CN_Date'] = result_df['CN_Date'].apply(
        lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else ''
    )
    
    result_df = result_df.sort_values(['Key', 'PN_Date', 'CN_Date'])
    result_df.to_csv('pcr_v2.csv', index=False)
    return result_df

if __name__ == "__main__":
    results = process_mapping()
    print("Mapping completed. Check complete_mapping.csv")

Mapping completed. Check complete_mapping.csv


### Version 3

In [7]:
def process_mapping():
    # Load data
    pn = pd.read_csv('pn_data.csv')
    cn = pd.read_csv('cn_data.csv')
    
    # Clean and convert dates
    pn['date'] = pd.to_datetime(pn['date'].replace('-', pd.NaT), format='%d-%m-%Y', errors='coerce')
    cn['vendor_cn_date'] = pd.to_datetime(cn['vendor_cn_date'].replace('-', pd.NaT), format='%Y-%m-%d', errors='coerce')
    
    # Remove invalid dates and sort
    pn = pn.dropna(subset=['date']).sort_values(['key', 'date'])
    cn = cn.dropna(subset=['vendor_cn_date']).sort_values(['key', 'vendor_cn_date'])
    
    mapped_records = []
    
    for key in pn['key'].unique():
        key_pns = pn[pn['key'] == key].reset_index(drop=True)
        key_cns = cn[cn['key'] == key].reset_index(drop=True)
        
        pn_qty_remaining = 0
        cn_qty_remaining = 0
        
        max_rows = max(len(key_pns), len(key_cns))
        
        for i in range(max_rows):
            has_pn = i < len(key_pns)
            has_cn = i < len(key_cns)
            
            if has_pn:
                pn_row = key_pns.iloc[i]
                pn_qty = pn_row['quantity'] + pn_qty_remaining
            else:
                pn_qty = pn_qty_remaining
            
            if has_cn:
                cn_row = key_cns.iloc[i]
                cn_qty = cn_row['billed_quantity'] + cn_qty_remaining
            else:
                cn_qty = cn_qty_remaining
            
            # Calculate mapping quantities
            qty_to_map = min(pn_qty, cn_qty) if (pn_qty > 0 and cn_qty > 0) else 0
            
            record = {
                'PN_Date': pn_row['date'] if has_pn else None,
                'CN_Date': cn_row['vendor_cn_date'] if has_cn else None,
                'Pick_Note': pn_row['pick_note_number'] if has_pn else None,
                'Credit_Note': cn_row['debit_note_number'] if has_cn else None,
                'Key': key,
                'Vendor': pn_row['vendor_name'] if has_pn else (cn_row['vendor_name'] if has_cn else None),
                'Original_PN_Qty': pn_row['quantity'] if has_pn else 0,
                'Original_CN_Qty': cn_row['billed_quantity'] if has_cn else 0,
                'PN_Qty': qty_to_map,
                'CN_Qty': qty_to_map,
                'PN_Remaining': pn_qty - qty_to_map,
                'CN_Remaining': cn_qty - qty_to_map
            }
            
            mapped_records.append(record)
            
            # Update remaining quantities
            pn_qty_remaining = pn_qty - qty_to_map if pn_qty - qty_to_map > 0 else 0
            cn_qty_remaining = cn_qty - qty_to_map if cn_qty - qty_to_map > 0 else 0
    
    # Create DataFrame and format dates
    result_df = pd.DataFrame(mapped_records)
    result_df['PN_Date'] = result_df['PN_Date'].apply(
        lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else None
    )
    result_df['CN_Date'] = result_df['CN_Date'].apply(
        lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else None
    )
    
    result_df.to_csv('mapping_results.csv', index=False)
    return result_df

# Execute mapping
results = process_mapping()

Mapping completed. Check complete_mapping.csv


### Version 3.2

### PERFECT SOLUTION

In [3]:
def add_remarks(pn_remaining, cn_remaining):
    if pn_remaining == cn_remaining:
        return f"Exact Match"
    
    elif pn_remaining > 0:
        return f"PN Exceeds"
    
    elif cn_remaining > 0:
        return f"CN Exceeds"

In [6]:
import pandas as pd

def process_mapping2():
    # Load and prepare data
    pn = pd.read_csv('pn_data.csv')
    cn = pd.read_csv('cn_data.csv')
    
    # Clean and convert dates
    pn['date'] = pd.to_datetime(pn['date'].replace('-', pd.NaT), format='%d-%m-%Y', errors='coerce')
    cn['vendor_cn_date'] = pd.to_datetime(cn['vendor_cn_date'].replace('-', pd.NaT), format='%Y-%m-%d', errors='coerce')
    
    # Remove invalid dates and sort 
    pn = pn.dropna(subset=['date']).sort_values(['key', 'date'])
    cn = cn.dropna(subset=['vendor_cn_date']).sort_values(['key', 'vendor_cn_date'])
    
    # An array for storing mapped records.

    mapped_records = []
    
    # Get all unique keys from both dataframes. Basically a set with all the possible keys
    all_keys = set(pn['key'].unique()) | set(cn['key'].unique())
    
    # for each key 
    for key in all_keys:

        # Key_pns is a dataframe which stores the entire data of pn corresponding to the current key. Similar is for cns
        key_pns = pn[pn['key'] == key].reset_index(drop=True)
        key_cns = cn[cn['key'] == key].reset_index(drop=True)
        
        # Handle key present in only one dataset
        if len(key_pns) == 0:
            # Key only in CN data
            for i in range(len(key_cns)):
                cn_row = key_cns.iloc[i]
                mapped_records.append({
                    'PN_Date': None,
                    'CN_Date': cn_row['vendor_cn_date'],
                    'Pick_Note': None,
                    'Credit_Note': cn_row['debit_note_number'],
                    'Key': key,
                    'Vendor': cn_row['vendor_name'],
                    'Original_PN_Qty': 0,
                    'Original_CN_Qty': cn_row['billed_quantity'],
                    'PN_Qty': 0,
                    'CN_Qty': cn_row['billed_quantity'],
                    'PN_Remaining': 0,  # Already 0 since no PN exists
                    'CN_Remaining': cn_row['billed_quantity']  # Set remaining to original qty
                })
            continue
            
        if len(key_cns) == 0:
            # Key only in PN data
            for i in range(len(key_pns)):
                pn_row = key_pns.iloc[i]
                mapped_records.append({
                    'PN_Date': pn_row['date'],
                    'CN_Date': None,
                    'Pick_Note': pn_row['pick_note_number'],
                    'Credit_Note': None,
                    'Key': key,
                    'Vendor': pn_row['vendor_name'],
                    'Original_PN_Qty': pn_row['quantity'],
                    'Original_CN_Qty': 0,
                    'PN_Qty': pn_row['quantity'],
                    'CN_Qty': 0,
                    'PN_Remaining': pn_row['quantity'],  # Set remaining to original qty
                    'CN_Remaining': 0  # Already 0 since no CN exists
                })
            continue
                
        # Rest of your existing mapping logic for when key exists in both datasets
        pn_qty_remaining = 0
        cn_qty_remaining = 0
        max_rows = max(len(key_pns), len(key_cns))
        
        for i in range(max_rows):
            has_pn = i < len(key_pns)
            has_cn = i < len(key_cns)
            
            if has_pn:
                pn_row = key_pns.iloc[i]
                pn_qty = pn_row['quantity'] + pn_qty_remaining
            else:
                pn_qty = pn_qty_remaining
            
            if has_cn:
                cn_row = key_cns.iloc[i]
                cn_qty = cn_row['billed_quantity'] + cn_qty_remaining
            else:
                cn_qty = cn_qty_remaining
            
            # Calculate mapping quantities
            qty_to_map = min(pn_qty, cn_qty) if (pn_qty > 0 and cn_qty > 0) else 0
            
            record = {
                'PN_Date': pn_row['date'] if has_pn else None,
                'CN_Date': cn_row['vendor_cn_date'] if has_cn else None,
                'Pick_Note': pn_row['pick_note_number'] if has_pn else None,
                'Credit_Note': cn_row['debit_note_number'] if has_cn else None,
                'Key': key,
                'Vendor': pn_row['vendor_name'] if has_pn else (cn_row['vendor_name'] if has_cn else None),
                'Original_PN_Qty': pn_row['quantity'] if has_pn else 0,
                'Original_CN_Qty': cn_row['billed_quantity'] if has_cn else 0,
                'PN_Qty': qty_to_map,
                'CN_Qty': qty_to_map,
                'PN_Remaining': pn_qty - qty_to_map,
                'CN_Remaining': cn_qty - qty_to_map
            }                                                                                                      
            
            mapped_records.append(record)
            
            # Update remaining quantities
            pn_qty_remaining = pn_qty - qty_to_map if pn_qty - qty_to_map > 0 else 0
            cn_qty_remaining = cn_qty - qty_to_map if cn_qty - qty_to_map > 0 else 0
    
    # Create DataFrame and format dates
    result_df = pd.DataFrame(mapped_records)
    result_df['PN_Date'] = result_df['PN_Date'].apply(
        lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else None
    )
    result_df['CN_Date'] = result_df['CN_Date'].apply(
        lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else None
    )

    # Now create a new column named Remarks which would have value in string format and take value depending on the function add_remarks
    result_df['Remarks'] = result_df.apply(lambda x: add_remarks(x['PN_Remaining'], x['CN_Remaining']), axis=1)
    
    result_df.to_csv('complete_mapping.csv', index=False)
    return result_df

results = process_mapping2()

In [7]:
import pandas as pd

def process_mapping2():
    # Load and prepare data
    pn = pd.read_csv('pn_data.csv')
    cn = pd.read_csv('cn_data.csv')
    
    # Clean and convert dates
    pn['date'] = pd.to_datetime(pn['date'].replace('-', pd.NaT), format='%d-%m-%Y', errors='coerce')
    cn['vendor_cn_date'] = pd.to_datetime(cn['vendor_cn_date'].replace('-', pd.NaT), format='%Y-%m-%d', errors='coerce')
    
    # Remove invalid dates and sort 
    pn = pn.dropna(subset=['date']).sort_values(['key', 'date'])
    cn = cn.dropna(subset=['vendor_cn_date']).sort_values(['key', 'vendor_cn_date'])
    
    # An array for storing mapped records.
    mapped_records = []
    
    # Get all unique keys from both dataframes. Basically a set with all the possible keys
    all_keys = set(pn['key'].unique()) | set(cn['key'].unique())
    
    # for each key 
    for key in all_keys:

        # Key_pns is a dataframe which stores the entire data of pn corresponding to the current key. Similar is for cns
        key_pns = pn[pn['key'] == key].reset_index(drop=True)
        key_cns = cn[cn['key'] == key].reset_index(drop=True)
        
        # Handle key present in only one dataset
        if len(key_pns) == 0:
            # Key only in CN data
            for i in range(len(key_cns)):
                cn_row = key_cns.iloc[i]
                mapped_records.append({
                    'PN_Date': None,
                    'CN_Date': cn_row['vendor_cn_date'],
                    'Pick_Note': None,
                    'Credit_Note': cn_row['debit_note_number'],
                    'Key': key,
                    'Vendor': cn_row['vendor_name'],
                    'Original_PN_Qty': 0,
                    'Original_CN_Qty': cn_row['billed_quantity'],
                    'PN_Qty': 0,
                    'CN_Qty': cn_row['billed_quantity'],
                    'PN_Remaining': 0,  # Already 0 since no PN exists
                    'CN_Remaining': cn_row['billed_quantity']  # Set remaining to original qty
                })
            continue
            
        if len(key_cns) == 0:
            # Key only in PN data
            for i in range(len(key_pns)):
                pn_row = key_pns.iloc[i]
                mapped_records.append({
                    'PN_Date': pn_row['date'],
                    'CN_Date': None,
                    'Pick_Note': pn_row['pick_note_number'],
                    'Credit_Note': None,
                    'Key': key,
                    'Vendor': pn_row['vendor_name'],
                    'Original_PN_Qty': pn_row['quantity'],
                    'Original_CN_Qty': 0,
                    'PN_Qty': pn_row['quantity'],
                    'CN_Qty': 0,
                    'PN_Remaining': pn_row['quantity'],  # Set remaining to original qty
                    'CN_Remaining': 0  # Already 0 since no CN exists
                })
            continue
                
        # Rest of your existing mapping logic for when key exists in both datasets
        pn_qty_remaining = 0
        cn_qty_remaining = 0
        max_rows = max(len(key_pns), len(key_cns))
        
        pick_notes = []
        credit_notes = []
        
        for i in range(max_rows):
            has_pn = i < len(key_pns)
            has_cn = i < len(key_cns)
            
            if has_pn:
                pn_row = key_pns.iloc[i]
                pn_qty = pn_row['quantity'] + pn_qty_remaining
                pick_notes.append(pn_row['pick_note_number'])
            else:
                pn_qty = pn_qty_remaining
            
            if has_cn:
                cn_row = key_cns.iloc[i]
                cn_qty = cn_row['billed_quantity'] + cn_qty_remaining
                credit_notes.append(cn_row['debit_note_number'])
            else:
                cn_qty = cn_qty_remaining
            
            # Calculate mapping quantities
            qty_to_map = min(pn_qty, cn_qty) if (pn_qty > 0 and cn_qty > 0) else 0
            
            record = {
                'PN_Date': pn_row['date'] if has_pn else None,
                'CN_Date': cn_row['vendor_cn_date'] if has_cn else None,
                'Pick_Note': pick_notes.copy() if has_pn else None,
                'Credit_Note': credit_notes.copy() if has_cn else None,
                'Key': key,
                'Vendor': pn_row['vendor_name'] if has_pn else (cn_row['vendor_name'] if has_cn else None),
                'Original_PN_Qty': pn_row['quantity'] if has_pn else 0,
                'Original_CN_Qty': cn_row['billed_quantity'] if has_cn else 0,
                'PN_Qty': qty_to_map,
                'CN_Qty': qty_to_map,
                'PN_Remaining': pn_qty - qty_to_map,
                'CN_Remaining': cn_qty - qty_to_map
            }
            
            mapped_records.append(record)
            
            # Update remaining quantities
            pn_qty_remaining = pn_qty - qty_to_map if pn_qty - qty_to_map > 0 else 0
            cn_qty_remaining = cn_qty - qty_to_map if cn_qty - qty_to_map > 0 else 0
            
            # Remove used pick notes and credit notes
            if pn_qty_remaining == 0:
                pick_notes = []
            if cn_qty_remaining == 0:
                credit_notes = []
    
    # Create DataFrame and format dates
    result_df = pd.DataFrame(mapped_records)
    result_df['PN_Date'] = result_df['PN_Date'].apply(
        lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else None
    )
    result_df['CN_Date'] = result_df['CN_Date'].apply(
        lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else None
    )

    # Now create a new column named Remarks which would have value in string format and take value depending on the function add_remarks
    result_df['Remarks'] = result_df.apply(lambda x: add_remarks(x['PN_Remaining'], x['CN_Remaining']), axis=1)
    
    result_df.to_csv('complete_mapping2.csv', index=False)
    return result_df

results = process_mapping2()